In [1]:
# save csv as data frame 

import pandas as pd
import numpy as np
import re


# Clean Up/ Data Wrangling

# Please note that within this dataset, Nan values will be treated as 0. 

In [2]:
demographics_csv_files = ["ACSDP1Y2010.DP05-Data.csv",
"ACSDP5Y2021.DP05-Data.csv",
"ACSDP1Y2011.DP05-Data.csv",
"ACSDP1Y2012.DP05-Data.csv",
"ACSDP1Y2013.DP05-Data.csv",
"ACSDP1Y2014.DP05-Data.csv",
"ACSDP1Y2015.DP05-Data.csv",
"ACSDP1Y2016.DP05-Data.csv",
"ACSDP1Y2017.DP05-Data.csv",
"ACSDP5Y2012.DP05-Data.csv",
"ACSDP5Y2011.DP05-Data.csv",
"ACSDP5Y2010.DP05-Data.csv",
"ACSDP1Y2021.DP05-Data.csv",
"ACSDP1Y2019.DP05-Data.csv",
"ACSDP1Y2018.DP05-Data.csv"]

In [3]:
def clean_census_df(_census_df, year):
    # remove all the weird values for estimates
    census_df = _census_df.replace({'**': np.nan, 
                           '-': np.nan,
                           '***': np.nan, 
                           '*****': np.nan, 
                           '(X)': np.nan, 
                           'N/A': np.nan,
                           'null': np.nan, 
                           'median-': np.nan, 
                           'median+': np.nan,
                          '-666666666': np.nan, 
                          'N': np.nan,
                          '-888888888': np.nan,
                          '-Varies': np.nan,
                          '-222222222': np.nan,
                          '-333333333': np.nan,
                          '*': np.nan,})
    census_df = census_df.dropna(how='all', axis='columns').dropna(how='all')
    
    if census_df.empty:
        return None
    
    print(year)
    # Re-index using the "Labels" Columns
    # Reference : https://api.census.gov/data/2021/acs/acs5/profile/groups/DP05.html 
    
    census_df.columns = pd.MultiIndex.from_frame(pd.MultiIndex.from_tuples(census_df.columns.str.split('!!').tolist()).to_frame().fillna(""))

    census_df = census_df.T.reset_index()
    
#     census_df.columns = pd.MultiIndex.from_frame(pd.MultiIndex.from_tuples(census_df.columns.str.split('!!').tolist()).to_frame().fillna(""))
#     census_df = census_df.T.reset_index()
    
    # Get relevant columns 
    census_df = census_df[census_df[0].str.contains('Zip Code') | census_df[0].str.contains('Estimate')]
    
    # Update the index and drop unnecessary columns
    multi_index_columns = census_df.apply(lambda x: x.str.isnumeric()==False).all()[census_df.apply(lambda x: x.str.isnumeric()==False).all()].index
    
    census_df.index = pd.MultiIndex.from_frame(census_df[multi_index_columns])

    census_df = census_df.drop(columns=census_df.index.names)    
    
    # Fill straggling NaNs
    
    census_df = census_df.fillna(0)
    
    # Add year
    census_df.loc[('Year'),:] = year
    
    census_df = census_df.T
    census_df = census_df.set_index(['Year','Zip Code'])
    
    census_df.columns = census_df.columns.droplevel(0)

    
    return census_df
    


In [4]:
valid_census_dfs = []
for file in demographics_csv_files:
    year = re.search("([0-9]{4})", file)[0]
    file_path = './data/raw/' + file
    df = pd.read_csv(file_path, header=1, dtype=str)
    temp_df = df
    temp_df = temp_df.loc[temp_df['Geographic Area Name'].str.contains('ZCTA')]
    temp_df['Zip Code'] = temp_df['Geographic Area Name'][temp_df['Geographic Area Name'].str.contains('ZCTA')].str.split(" ").str[1]
    
    df = clean_census_df(temp_df, year)
    
    if df is None: 
        continue
    else: 
#         print(file)
#         df = clean_census_df(temp_df, year)
        valid_census_dfs.append(df)
        
len(valid_census_dfs)



/var/folders/ht/q4d_vppn3x5d3_p7bs499mdh0000gn/T/ipykernel_85377/2348089967.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['Zip Code'] = temp_df['Geographic Area Name'][temp_df['Geographic Area Name'].str.contains('ZCTA')].str.split(" ").str[1]


2021


/var/folders/ht/q4d_vppn3x5d3_p7bs499mdh0000gn/T/ipykernel_85377/2348089967.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['Zip Code'] = temp_df['Geographic Area Name'][temp_df['Geographic Area Name'].str.contains('ZCTA')].str.split(" ").str[1]


2012


/var/folders/ht/q4d_vppn3x5d3_p7bs499mdh0000gn/T/ipykernel_85377/2348089967.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['Zip Code'] = temp_df['Geographic Area Name'][temp_df['Geographic Area Name'].str.contains('ZCTA')].str.split(" ").str[1]


2011


3

In [5]:
valid_census_dfs[0].columns = valid_census_dfs[0].columns.droplevel(0)

In [6]:
df = pd.concat(valid_census_dfs, axis=1).fillna(0)
df.columns = pd.MultiIndex.from_tuples(df.columns.tolist())
df=df.reset_index()

/var/folders/ht/q4d_vppn3x5d3_p7bs499mdh0000gn/T/ipykernel_85377/772371714.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  df=df.reset_index()
/var/folders/ht/q4d_vppn3x5d3_p7bs499mdh0000gn/T/ipykernel_85377/772371714.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  df=df.reset_index()


In [7]:
# make df to csv 
df.to_csv('./data/clean/demographics.csv')

# Task: Figure out if we should omit certain missing zip codes using census data 

In [18]:
filepath = './stats/zipcode_data/missing_home_rent_values/missing_zipcode_home_values.csv'
missing_df = pd.read_csv(filepath, dtype=str)
missing_df["zipcode"].drop_duplicates()

0      10020
8      10037
12     10115
20     10301
23     10302
31     10303
39     10304
47     10305
55     10306
63     10307
71     10308
79     10309
86     10310
94     10311
95     10312
103    10314
111    10474
119    10550
125    10704
128    10705
135    10803
136    11001
141    11005
148    11040
149    11096
151    11109
159    11351
167    11359
174    11371
182    11430
186    11580
188    11693
196    11697
Name: zipcode, dtype: object

In [23]:
filepath = './stats/zipcode_data/missing_home_rent_values/missing_zipcode_home_values.csv'
missing_df2 = pd.read_csv(filepath, dtype=str)


0      10020
8      10037
12     10115
20     10301
23     10302
31     10303
39     10304
47     10305
55     10306
63     10307
71     10308
79     10309
86     10310
94     10311
95     10312
103    10314
111    10474
119    10550
125    10704
128    10705
135    10803
136    11001
141    11005
148    11040
149    11096
151    11109
159    11351
167    11359
174    11371
182    11430
186    11580
188    11693
196    11697
Name: zipcode, dtype: object

In [46]:
missing_zip_codes = pd.concat([missing_df["zipcode"],missing_df2["zipcode"]]).drop_duplicates()
missing_zip_codes

0      10020
8      10037
12     10115
20     10301
23     10302
31     10303
39     10304
47     10305
55     10306
63     10307
71     10308
79     10309
86     10310
94     10311
95     10312
103    10314
111    10474
119    10550
125    10704
128    10705
135    10803
136    11001
141    11005
148    11040
149    11096
151    11109
159    11351
167    11359
174    11371
182    11430
186    11580
188    11693
196    11697
Name: zipcode, dtype: object

# All of the "missing" zip codes exist in the demographics census data

In [48]:
missing_zip_codes[~missing_zip_codes.isin(df["Zip Code"])]

Series([], Name: zipcode, dtype: object)

In [51]:
missing_df = df[df["Zip Code"].isin(missing_zip_codes)]
missing_df

Year Zip Code Total population                \
                                       Male Female   
                                                     
                                                     
18    2021    10020                0      0      0   
35    2021    10037            19374   8533  10841   
47    2021    10115                0      0      0   
70    2021    10301            38736  18528  20208   
71    2021    10302            18144   9592   8552   
...    ...      ...              ...    ...    ...   
3943  2011    11371                0      0      0   
3971  2011    11430                0      0      0   
4013  2011    11580                0      0      0   
4020  2011    11693                0      0      0   
4022  2011    11697                0      0      0   

                                                                   \
     Sex ratio (males per 100 females) Under 5 years 5 to 9 years   
                                                                    
                                                                    
18                                   0             0            0   
35                                78.7          1286          994   
47                                   0             0            0   
70                                91.7          2244         1881   
71                               112.2          1364         1170   
...                                ...           ...          ...   
3943                                 0             0            0   
3971                                 0             0            0   
4013                                 0             0            0   
4020                                 0             0            0   
4022                                 0             0            0   

                                    ... HISPANIC OR LATINO AND RACE  \
     10 to 14 years 15 to 19 years  ...      Not Hispanic or Latino   
                                    ...                 White alone   
                                    ...                               
18                0              0  ...                           0   
35             1065            473  ...                           0   
47                0              0  ...                           0   
70             2020           2442  ...                           0   
71             1295           1180  ...                           0   
...             ...            ...  ...                         ...   
3943              0              0  ...                           0   
3971              0              0  ...                           6   
4013              0              0  ...                       13691   
4020              0              0  ...                        5568   
4022              0              0  ...                        4305   

                                                                              \
                                                                               
     Black or African American alone American Indian and Alaska Native alone   
                                                                               
18                                 0                                       0   
35                                 0                                       0   
47                                 0                                       0   
70                                 0                                       0   
71                                 0                                       0   
...                              ...                                     ...   
3943                               0                                       0   
3971                              26                                       0   
4013                           11418                                      11   
4020                            3580                 

In [134]:
missing_df = missing_df.set_index(['Zip Code', 'Year']).sort_index()

/var/folders/ht/q4d_vppn3x5d3_p7bs499mdh0000gn/T/ipykernel_85377/1154591959.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  missing_df = missing_df.set_index(['Zip Code', 'Year']).sort_index()


In [221]:
# missing_df[missing[df]]

total_population = missing_df["Total population"]
total_population = total_population.astype(float)

total_population_missing_zc = total_population.groupby(level="Zip Code").sum()
total_population_missing_zc.columns = ["-".join(c) for c in total_population_missing_zc.columns.to_flat_index()]

total_population_missing_zc = total_population_missing_zc.sum(axis=1)

total_population_missing_zc[total_population_missing_zc ==0]

Zip Code
10020    0.0
10115    0.0
10311    0.0
11351    0.0
11359    0.0
11371    0.0
dtype: float64

In [223]:
total_housing_units = missing_df["Total housing units"]
total_housing_units.columns = [1,2]
total_housing_units["Total housing units"] = total_housing_units[1].astype(int) + total_housing_units[2].astype(int)
total_housing_units = total_housing_units["Total housing units"]

missing_zc = total_housing_units[total_housing_units==0].reset_index().groupby("Zip Code").count().sort_values(by="Year", ascending=False)
missing_zc[missing_zc["Year"] == max(missing_zc["Year"])].astype(float)


/var/folders/ht/q4d_vppn3x5d3_p7bs499mdh0000gn/T/ipykernel_85377/101495429.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_housing_units["Total housing units"] = total_housing_units[1].astype(int) + total_housing_units[2].astype(int)


,Year,Total housing units
Zip Code,,
10020,3.0,3.0
11371,3.0,3.0
10115,3.0,3.0
10311,3.0,3.0
11359,3.0,3.0
11430,3.0,3.0


*Explanation of Zip Code Omissions*

When we were merging our rerntal data sets, we noticed that there were a few zip codes that were listed inside of our data for Air BnB were missing from the Zillow data set. To investigate this further, we used demographics data to narrow the zip codes 
10020 can be excluded because it only holds Rockefeller Plaza. 
10115 can be excluded because it only holds Riverside Church within its boundaries. 
10311 is just a school and a youth complex. 
11351 is just a sliver of a street in Queens that has a radius of only ~200 yard radius. 
11359 is a peninsula on Queens that contains a park, some NYC recreational buildings, but no residential buildings.
11371 holds LaGuardia Airport and Rikers Island. 
